In [ ]:
import time
import re
import pandas as pd
import numpy as np
import requests
import pickle
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

입력어 받는 파이썬파일로.  
원하는리뷰수를 인풋?  
-이거계산하는거만들어보고  
url도 인풋으로

In [74]:
#스크롤 횟수

def Youtube_crawl(url, scroll_cnt):
    scroll_cnt = scroll_cnt

    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(2) #댓글창로딩

    body = driver.find_element_by_tag_name('body')

    while scroll_cnt:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        scroll_cnt -= 1

    reviews = driver.find_elements_by_xpath('//*[@id="main"]')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
    print(scroll_cnt, len(reviews))


    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list

page_down을 사용하면 한번에 사이트의 맨 아래로 갈 수 없기에 느리다.

In [86]:
url = 'https://www.youtube.com/watch?v=HhgcXuN0MlU&t=599s'

#자신의 크롬 버전에 맞는 드라이버 다운
driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
driver.get(url)

time.sleep(5) #댓글창로딩

scroll_cnt = 20

for i in range(scroll_cnt):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
    time.sleep(5)
    
# last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

# while True:
#     driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
#     time.sleep(5)
#     new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

#     if new_page_height == last_page_height:
#         break

# 댓글 하나하나 쭈욱 처리하면서 내려오는게 더 낫지않을까

In [221]:
reply = driver.find_elements_by_xpath('//*[@id="contents"]/ytd-comment-thread-renderer')

In [223]:
len(reply)

420

In [108]:
#답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

for load in load_detail:
    if load.text == '자세히 보기':
        load.click()

In [197]:
load_reply = driver.find_elements_by_css_selector('#sections #text')

In [205]:
for idx, t in enumerate(test):
    if '보기' in t.text:
        print(idx)

2
18
21
24
27
30
33
36
40
43
46
53
63
66
73
80
91
99
102
105
111
116
119
122
126
131
154
165
175
181
188
193
197
201
206
209
221
253
258
273
280
296
307
327
350
382
416
419
422
473
482
495
502
511
516
527


In [209]:
more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

In [216]:
more_replies = []
for reply in more_reply:
    more_replies.append(reply.text)
    
t_list = []
for t in test:
    t_list.append(t.text)

위에서 아래로 내려오면서 한번씩 클릭하고, 답글 더보기가 없을때 멈추자.  
비효율적인것은 안다.

# 클릭을하고나서도 답글더보기, 자세히보기가나온다
- 답글 더보기로 먼저 다 출력하고나서 자세히보기는 그다음에 진행하는걸로

In [78]:
def Youtube_crawl_2(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩

    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    #댓글 최하단까지 이동
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height
    
    #자세히 보기를 클릭해 리뷰 잘림 현상 방지
    load_more = driver.find_elements_by_xpath('//*[@id="more"]/span')

    for load in load_more:
        if load.text == '자세히 보기':
            load.click()

    reviews = driver.find_elements_by_xpath('//*[@id="main"]')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')

    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list
    
    return df

In [79]:
df = Youtube_crawl_2('https://www.youtube.com/watch?v=HhgcXuN0MlU&t=599s')

스크롤횟수를 변화시키면서 리뷰가 어느정도씩 추가되는지를 봐보자

### 좋아요 수 전처리
- '1천', or ''

답글과 자세히보기문제

In [55]:
df['좋아요'] = df['좋아요'].apply(lambda x: re.sub('천 | 만','000',x)) #천의 경우는 1천 / 만의 경우는 1.1만 의 경우로 표현됨

In [56]:
df['좋아요']

0        1419111
1        1317131
2        1117171
3      111010101
4        1118181
         ...    
375        17161
376            1
377            1
378            1
379            1
Name: 좋아요, Length: 380, dtype: object

like에 가중치를 둔다면?
댓글 * like로 가중치를두어계산(이때 log2 씌워주고) #like가없다면 차이를두어야겠지?